# Question sur les clusters

## Python stuff

In [1]:
import math
import numpy as np
import geopandas as gpd
import contextily as ctx
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from s3_utils import S3Manager

# Configs
_ = plt.style.use("ggplot")

## 1 - Récupération des données

In [2]:
# Chargement des éléments de connexion sur S3
s3_manager = S3Manager()
bucket = "fub-s3"

In [3]:
# Déjà executé
# s3_manager.download_from_s3(
#     bucket,
#     "data/DFG/2025/data_geo/données-carto-2025-06-03.geojson", 
#     "temp/données-carto-2025-06-03.geojson"
# )

In [4]:
# Données données internes
# gdf = s3_manager.load_geojson_from_s3(bucket, "data/DFG/2025/data_geo/données-carto-2025-06-03.geojson")
gdf = gpd.read_file("temp/données-carto-2025-06-03.geojson")

In [5]:
# Récupération par communes
communes = gpd.read_file("https://france-geojson.gregoiredavid.fr/repo/communes.geojson")
communes = communes.to_crs(4326)

In [6]:
gdf = gdf.to_crs(communes.crs)
gdf_filtre = gpd.sjoin(gdf, communes, predicate="within", how="inner")
gdf_filtre = gdf_filtre.drop(columns=["index_right"])
gdf_filtre.rename(columns={"code": "code_commune", "nom": "nom_commune"}, inplace=True)

## 2 - % total de point contenu dans les clusters réalisés et ceux qui n'y sont pas.

In [7]:
# Fonction de clustering
def dbscan_clustering(gdf: gpd.GeoDataFrame, dist_m=50):
    """
    Effectue un clustering spatial sur un GeoDataFrame.
    
    :param gdf: GeoDataFrame avec des géométries de points.
    :param dist_m : Distance en mètre entre les points pour former un cluster.
    :return: GeoDataFrame avec les clusters ajoutés.
    """
    # Step --> Reprojection
    gdf = gdf.to_crs(epsg=2154)
    print(f"✅ La projection du GeoDataFrame est : {gdf.crs}")

    # Step calcul log(n)
    min_sample = round(math.log(gdf.shape[0])) + 1
    print(f"ℹ️ La taille de l'échantillon est : {gdf.shape[0]}")
    print(f"ℹ️ Le nombre minimal d'échantillons pour le clustering est : {min_sample}")

    # Appliquer le clustering
    dbscan = DBSCAN(eps=dist_m, min_samples=min_sample, metric='euclidean')
    dbscan.fit(gdf.geometry.apply(lambda geom: (geom.x, geom.y)).tolist())
    gdf["cluster_id"] = dbscan.labels_
    gdf = gdf[gdf["cluster_id"] != -1] # elimine le bruit

    print(f"ℹ️ Le nombre de clusters formés est : {gdf['cluster_id'].nunique()}")

    return gdf

### Cluster 58 : points rouges

In [8]:
gdf_58 = gdf_filtre[gdf_filtre["cat"] == 58].copy()

In [9]:
gdf_58_cluster =  dbscan_clustering(gdf_58, dist_m=50)

✅ La projection du GeoDataFrame est : EPSG:2154
ℹ️ La taille de l'échantillon est : 578129
ℹ️ Le nombre minimal d'échantillons pour le clustering est : 14
ℹ️ Le nombre de clusters formés est : 2554


In [10]:
print(f"Nombre de point rouges compris dans les clusters : {gdf_58_cluster.shape[0]}")
print(f"Nombre total de point rouges : {gdf_58.shape[0]}")
print(f"Nombre total de point rouges non compris dans les clusters : {gdf_58.shape[0] - gdf_58_cluster.shape[0]}")
print(f"Proportion de points rouges dans les clusters : {gdf_58_cluster.shape[0] / gdf_58.shape[0] * 100:.2f}%")

Nombre de point rouges compris dans les clusters : 109061
Nombre total de point rouges : 578129
Nombre total de point rouges non compris dans les clusters : 469068
Proportion de points rouges dans les clusters : 18.86%


### Cluster 59 : points bleues

In [11]:
gdf_59 = gdf_filtre[gdf_filtre["cat"] == 59].copy()

In [12]:
gdf_59_cluster =  dbscan_clustering(gdf_59, dist_m=50)

✅ La projection du GeoDataFrame est : EPSG:2154
ℹ️ La taille de l'échantillon est : 289602
ℹ️ Le nombre minimal d'échantillons pour le clustering est : 14
ℹ️ Le nombre de clusters formés est : 1344


In [13]:
print(f"Nombre de point bleues compris dans les clusters : {gdf_59_cluster.shape[0]}")
print(f"Nombre total de point bleues : {gdf_59.shape[0]}")
print(f"Nombre total de point bleues non compris dans les clusters : {gdf_59.shape[0] - gdf_59_cluster.shape[0]}")
print(f"Proportion de points bleues dans les clusters : {gdf_59_cluster.shape[0] / gdf_59.shape[0] * 100:.2f}%")

Nombre de point bleues compris dans les clusters : 55573
Nombre total de point bleues : 289602
Nombre total de point bleues non compris dans les clusters : 234029
Proportion de points bleues dans les clusters : 19.19%


### Cluster 60 : points verts

In [14]:
gdf_60 = gdf_filtre[gdf_filtre["cat"] == 60].copy()

In [15]:
gdf_60_cluster =  dbscan_clustering(gdf_60, dist_m=50)

✅ La projection du GeoDataFrame est : EPSG:2154
ℹ️ La taille de l'échantillon est : 239552
ℹ️ Le nombre minimal d'échantillons pour le clustering est : 13
ℹ️ Le nombre de clusters formés est : 930


In [16]:
print(f"Nombre de point vert compris dans les clusters : {gdf_60_cluster.shape[0]}")
print(f"Nombre total de point vert : {gdf_60.shape[0]}")
print(f"Nombre total de point vert non compris dans les clusters : {gdf_60.shape[0] - gdf_60_cluster.shape[0]}")
print(f"Proportion de points vert dans les clusters : {gdf_60_cluster.shape[0] / gdf_60.shape[0] * 100:.2f}%")

Nombre de point vert compris dans les clusters : 48873
Nombre total de point vert : 239552
Nombre total de point vert non compris dans les clusters : 190679
Proportion de points vert dans les clusters : 20.40%


-- END --